In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import datetime as dt

In [2]:
df = pd.read_parquet("../Proyecto-Taxis-NYC/data/yellow_t_2018-01.parquet", engine="pyarrow")

In [4]:
#esto se uso inicilamente para poder darnos una idea de los datos a tratar

"""
def get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):
    labels,values,percent = list(),list(),list()
    if df.isna().sum().sum()>0:
        for column in df.columns:
            if df[column].isna().sum():
                labels.append(column)
                values.append(df[column].isna().sum())
                percent.append((df[column].isna().sum() / df.shape[0]) * 100)
        #Make a dataframe 
        missings=pd.DataFrame({'Colúmnas':labels,'Valores faltantes':values,'PorcentajePerdido':percent}).sort_values(by='PorcentajePerdido',ascending=False)
        

        if plot:
            plt.figure(figsize=figsize)
            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title('Porcentaje de valores perdidos',size=22)
            for i in plot.patches:
                plot.annotate(i.get_height(),
                (i.get_x() + i.get_width() / 2, i.get_height()),
                ha = 'center', va = 'baseline', fontsize = 12,
                color = 'black', xytext = (0,1),
                textcoords = 'offset pixels')
            locs, labels = plt.xticks()
            plt.setp(labels, rotation=rotation,size=16)
            plt.xlabel('Colúmnas',size=18)
            plt.ylabel('Porcentaje',size=18)
            plot.set_title('Porcentaje de valores perdidos',size=18)

        return missings
    else:
        return False
        """

#get_missings(df)

'\ndef get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):\n    labels,values,percent = list(),list(),list()\n    if df.isna().sum().sum()>0:\n        for column in df.columns:\n            if df[column].isna().sum():\n                labels.append(column)\n                values.append(df[column].isna().sum())\n                percent.append((df[column].isna().sum() / df.shape[0]) * 100)\n        #Make a dataframe \n        missings=pd.DataFrame({\'Colúmnas\':labels,\'Valores faltantes\':values,\'PorcentajePerdido\':percent}).sort_values(by=\'PorcentajePerdido\',ascending=False)\n        \n\n        if plot:\n            plt.figure(figsize=figsize)\n            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title(\'Porcentaje de valores perdidos\',size=22)\n            for i in plot.patches:\n                plot.annotate(i.get_height(),\n                (i.get_x() + i.get_width() / 2, i.get_height()),\n          

Aplicamos la transformacion de los tipos de datos en el dataset de taxis para una mejor categorizacion y eliminamos las columnas irrelevantes.

In [11]:
def t_datime(df):
    df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
    df["hora"]= df.tpep_pickup_datetime.dt.floor("H")

    df.drop(columns=["congestion_surcharge","airport_fee","VendorID","store_and_fwd_flag"], inplace=True)
    return df

def t_int(df):
    df['passenger_count'] = pd.to_numeric(df['passenger_count'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast="integer" )
    df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast="integer" )
    df['payment_type'] = pd.to_numeric(df['payment_type'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    return df

def t_float(df):
    df['trip_distance'] = pd.to_numeric(df['trip_distance'], downcast="float" )
    df['fare_amount'] = pd.to_numeric(df['fare_amount'], downcast="float" )
    df['extra'] = pd.to_numeric(df['extra'], downcast="float" )
    df['mta_tax'] = pd.to_numeric(df['mta_tax'], downcast="float" )
    df['tip_amount'] = pd.to_numeric(df['tip_amount'], downcast="float" )
    df['tolls_amount'] = pd.to_numeric(df['tolls_amount'], downcast="float" )
    df['improvement_surcharge'] = pd.to_numeric(df['improvement_surcharge'], downcast="float" )
    df['total_amount'] = pd.to_numeric(df['total_amount'], downcast="float" )
    return df

def total_transform(df):
    df = t_datime(df)
    df = t_int(df)
    df = t_float
    return df

df =  total_transform(df)


Creamos una mascara del dataframe cada 3 dias

In [ ]:
df_Impar = df[df.dayofmonth % 3 == 0]
#df_Impar = df_Impar.copy()

Eliminamos las columnas que consideramos insignificantes

In [6]:
#df_Impar.drop(columns=["congestion_surcharge","airport_fee","VendorID","store_and_fwd_flag"], inplace=True)

Sorteamos los value por date y filtramos los que estan fuera del mes

In [9]:
def sorted(df):
    global bol
    bol = 1
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    if bol == 1:
        df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']
        bol = 0
    return df

df_Impar = sorted(df_Impar)

In [12]:
df_Impar.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
644899,2,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,N,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3
599145,2,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,N,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3
480025,1,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,N,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3
478103,1,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,N,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3
469218,2,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,N,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3


De ahora en mas df_Impar es el df a usar

In [14]:
df_Impar.shape

(2908992, 18)

Vamos a obtener los Boroughs de salida y de llegada.

In [16]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

In [17]:
df_Impar.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan


Trabajamos con el clima

In [18]:
#documentacion mismo nombre del clima en cada update

clima = pd.read_csv("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/new york 2018-01-01 to 2018-02-01.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]

Transformamos los datos de clima para optimizar rendimiento y chequear que esten bien los formatos

In [ ]:
def t_clima():
    clima['temp'] = pd.to_numeric(clima['temp'], downcast="float" )
    clima['feelslike'] = pd.to_numeric(clima['feelslike'], downcast="float" )
    clima['precip'] = pd.to_numeric(clima['precip'], downcast="float" )
    clima['snow'] = pd.to_numeric(clima['snow'], downcast="float" )
    clima['snowdepth'] = pd.to_numeric(clima['snowdepth'], downcast="float" )
    clima["conditions"]=clima["conditions"].astype(str)
    return clima

clima = t_clima()

Aplicamos restriccion por meses dentro del clima

In [ ]:
def control_clima(dat):
    #aplicar sort fecha
    clima.sort_values(by='datetime', ascending=True, inplace=True)
    date = dat
    if (clima["datetime"] >= date):
        clima = clima[clima["datetime"] >= date]
        date_next= date + datetime.timedelta(months=+1)
        clima = clima[clima["datetime"] <= date_next]
        return clima
    else:
        date+= datetime.timedelta(months=+1)
        return control_clima(dat)

clima = control_clima(dt(2018,1,1))

Creamos columna auxiliar de hora dentro de la tabla clima

In [20]:
#clima in global

def aux (df_Impar):
    df_Impar["hora"]= df_Impar.tpep_pickup_datetime.dt.floor("H")
    clima.datetime = clima['datetime'].apply(pd.to_datetime)
    clima["hora"]= clima['datetime'].dt.floor("H")

clima = aux(df_Impar)

In [21]:
#cambiamos el nombre para poder hacer el merge
#zona.rename(columns={"datetime": "tpep_pickup_datetime"},inplace=True)

In [22]:
df_Impar.shape

(2908992, 19)

Se mergearon los datos del clima al df_Impar

In [23]:
df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

In [26]:
def millas(df):
  df['km_dollar'] = df['trip_distance']/df['fare_amount']
  return df
  

In [ ]:

def arreglo (df_Impar, ID):
    dfsinceros=df_Impar[df_Impar.fare_amount != 0]
    dfsinceros=dfsinceros[dfsinceros.fare_amount>0]
    dfsinceros = dfsinceros[dfsinceros.trip_distance != 0]

    df = dfsinceros[dfsinceros.RatecodeID == ID]
    df = millas(df)
    return df

df1 = arreglo(df_Impar, 1)
df2 = arreglo(df_Impar, 2)
df3 = arreglo(df_Impar, 3)
df4 = arreglo(df_Impar, 4)
df5 = arreglo(df_Impar, 5)


In [28]:
avg_km1 =df1['km_dollar'].mean()
avg_km2 =df2['km_dollar'].mean()
avg_km3 =df3['km_dollar'].mean()
avg_km4 =df4['km_dollar'].mean()
avg_km5 =df5['km_dollar'].mean()

Se borran los que tienen cero en trip distance y fare amount

In [30]:

df_Impar["borrar"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]==0  and row["trip_distance"]==0 else 0, axis=1)
df_Impar = df_Impar.drop(df_Impar[df_Impar.borrar == 1].index)
df_Impar = df_Impar.drop(columns="borrar")

In [1]:
def kms(df_Impar, ID, avg_km):
    df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km if row["RatecodeID"]==ID  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
    return df_Impar["trip_distance"]

df_Impar["trip_distance"] = kms(df_Impar, 1, avg_km1)
df_Impar["trip_distance"] = kms(df_Impar, 2, avg_km2)
df_Impar["trip_distance"] = kms(df_Impar, 3, avg_km3)
df_Impar["trip_distance"] = kms(df_Impar, 4, avg_km4)
df_Impar["trip_distance"] = kms(df_Impar, 5, avg_km5)


NameError: name 'df_Impar' is not defined

Se borran las ID innecesarias y los valores negativos, dado que tienen algunas incongruencias en algunas fechas por lo cual no podemos afirmar que son cancelaciones

In [ ]:
def neg_values(df_Impar):   
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==6].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==99].index)
    return df_Impar

df_Impar=neg_values(df_Impar)

Buscamos los outliers dentro de las columnas que pueden presentar tales

In [ ]:
def find_outliers_IQR(df):

   q1=df.quantile(0.01)

   q3=df.quantile(0.99)

   IQR=q3-q1

   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

def outliers_description(df_column):
    outliers = find_outliers_IQR(df_column)

    print("number of outliers: "+ str(len(outliers)))

    print("max outlier value: "+ str(outliers.max()))

    print("min outlier value: "+ str(outliers.min()))

    return outliers

outliers_description(df_Impar["trip_distance"])
outliers_description(df_Impar["fare_amount"])
outliers_description(df_Impar["tolls_amount"])
outliers_description(df_Impar["total_amount"])

Hasta ahora, se borraron las filas con trip distance y fare amount en cero, y se imputo la distancia recorrida en los campos correspondientes segun su Ratecode ID. restaria borrar los Ratecode ID 6 y 99 que son practicamente irrelevantes.

vamos a revisar y decidir que hacer con los precios negativos

Generamos csv para tablas sql

In [33]:
#df_Impar.to_csv('df_ImparSQL.csv', sep=',')

In [39]:
df_Impar[df_Impar.fare_amount<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
43,2018-01-03 00:00:27,2018-01-03 00:01:48,5,0.240000,1,79,79,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
6370,2018-01-03 01:37:09,2018-01-03 01:37:39,1,0.010000,1,132,132,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,3,Queens,Queens,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
6919,2018-01-03 01:52:56,2018-01-03 01:54:08,2,0.470000,1,48,48,3,-3.5,-0.5,-0.5,0.0,0.0,-0.3,-4.8,3,Manhattan,Manhattan,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
8010,2018-01-03 02:31:10,2018-01-03 02:33:12,2,0.450000,1,263,263,3,-4.0,-0.5,-0.5,0.0,0.0,-0.3,-5.3,3,Manhattan,Manhattan,2018-01-03 02:00:00,2018-01-03 02:00:00,-7.4,-12.4,0.0,0.0,0.25,Clear


In [40]:
df_Impar[df_Impar.trip_distance<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
11905,2018-01-03 05:24:43,2018-01-03 05:25:19,1,-732.619596,5,48,48,3,-34.8,0.0,-0.5,0.0,0.0,-0.3,-35.6,3,Manhattan,Manhattan,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
12080,2018-01-03 05:29:10,2018-01-03 05:29:53,1,-4.954313,3,129,129,4,-20.0,-0.5,0.0,0.0,0.0,-0.3,-20.8,3,Queens,Queens,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
13177,2018-01-03 05:49:31,2018-01-03 05:50:42,4,-0.614998,1,230,230,4,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
19874,2018-01-03 06:50:34,2018-01-03 06:50:42,2,-0.512498,1,132,132,4,-2.5,0.0,-0.5,0.0,0.0,-0.3,-3.3,3,Queens,Queens,2018-01-03 06:00:00,2018-01-03 06:00:00,-8.5,-8.5,0.0,0.0,0.08,Partially cloudy


In [38]:
df_Impar[df_Impar.total_amount<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
43,2018-01-03 00:00:27,2018-01-03 00:01:48,5,0.240000,1,79,79,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
6370,2018-01-03 01:37:09,2018-01-03 01:37:39,1,0.010000,1,132,132,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,3,Queens,Queens,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
6919,2018-01-03 01:52:56,2018-01-03 01:54:08,2,0.470000,1,48,48,3,-3.5,-0.5,-0.5,0.0,0.0,-0.3,-4.8,3,Manhattan,Manhattan,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
8010,2018-01-03 02:31:10,2018-01-03 02:33:12,2,0.450000,1,263,263,3,-4.0,-0.5,-0.5,0.0,0.0,-0.3,-5.3,3,Manhattan,Manhattan,2018-01-03 02:00:00,2018-01-03 02:00:00,-7.4,-12.4,0.0,0.0,0.25,Clear


In [26]:
df_Impar.loc[df_Impar['tpep_pickup_datetime'] == "2018-01-27 10:59:14"]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
2390960,2018-01-27 10:59:14,2018-01-27 11:20:14,1,1.00,1,234,186,1,13.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390961,2018-01-27 10:59:14,2018-01-27 11:07:50,1,1.70,1,236,151,1,8.5,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390962,2018-01-27 10:59:14,2018-01-27 11:21:12,1,4.35,1,236,246,2,18.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390963,2018-01-27 10:59:14,2018-01-27 11:21:08,1,3.34,1,148,246,1,16.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390964,2018-01-27 10:59:14,2018-01-27 11:02:01,1,0.40,1,262,263,1,4.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390965,2018-01-27 10:59:14,2018-01-27 11:10:44,1,2.77,1,48,43,2,11.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390966,2018-01-27 10:59:14,2018-01-27 11:06:16,1,1.61,1,237,170,1,7.5,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390967,2018-01-27 10:59:14,2018-01-27 11:01:22,1,0.77,1,237,236,2,4.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390968,2018-01-27 10:59:14,2018-01-27 11:09:15,3,1.01,1,90,68,2,8.0,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390969,2018-01-27 10:59:14,2018-01-27 11:08:39,1,1.50,99,238,142,1,8.5,0.0,...,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy


In [27]:
df_Impar.RatecodeID.unique()

array([ 1,  2,  5,  4,  3, 99,  6], dtype=int64)

In [28]:
df_Impar.drop(columns=["PULocationID","DOLocationID","datetime","hora"], inplace=True)
#df_Impar.drop(columns=["PULocationID","DOLocationID","datetime","hora"], inplace=True)